In [1]:
import pandas as pd
import os

In [2]:
basePath = '/Volumes/backup_128G/z_repository/Yang_data/學測指考英文'
dataPath = 'data_docx'
outputPath = 'output'

from_dataPath = '{0}/{1}'.format(basePath, dataPath)
to_dataPath = '{0}/{1}'.format(basePath, outputPath)

# Only care of levels 5 and 6 as high level

In [3]:
levels = [2, 3, 4, 5, 6, 9]
highlevels = [5, 6]

# 讀檔；去掉非英文字，回傳 word 字串，包含引用次數

In [4]:
# pip install python-docx 
import docx

def isAlpha(word):
    try:
        return word.encode('ascii').isalpha()
    except UnicodeEncodeError:
        return False
    
def isEnglish(s):
    return s.isalpha()
    
def wordCount(wordList):
    wCount = {}
    for word in wordList:
        if word not in wCount:
            wCount[word] = 1
        else:
            wCount[word] += 1
    return wCount
    
def split2Words(txt):
    wordList = []
    splitWords = txt.split()
    for word in splitWords:
        if isAlpha(word) == True:
            wordList.append(word)
        elif word != '':
            print(word, ' includes un-alpha characters.')
            
    return wordCount(wordList)
    
def replaceMultiple(mainString, toBeReplaces, newString):
#     outString = copy.copy(mainString)
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return mainString

def readTxt(filename):
    fullText = []
    # #VINCENT#, 20190917, To highlight the high level words
    plainTxt = []
    doc = docx.Document(filename)
    wordCnt = 0
    for paragraph in doc.paragraphs:
#         print(wordCnt, paragraph.text)
        if len(paragraph.text.replace(' ', '')) == 0:
            continue
        wordCnt += len(paragraph.text.split(' '))
        
        # #VINCENT#, 20190917, To highlight the high level words
        plainTxt.append(paragraph.text)
        processedPara = replaceMultiple(paragraph.text,
                                        ['…', '’', '‘', '\'', '-', '—', '－', '”', '“', '/',
                                         ';', '!', '.', '?', ',', ':', ':', '(', ')', '–'] , ' ')
        if len(processedPara) == 0:
            continue
        fullText.append(processedPara)

#     print(wordCnt)
    fullText = ' '.join(fullText)
    # #VINCENT#, 20190917, To highlight the high level words
    return wordCnt, plainTxt, split2Words(fullText)

# Parse Directory

In [5]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_COLOR_INDEX
from docx.enum.dml import MSO_THEME_COLOR
from docx.shared import RGBColor

fontColorMap = [WD_COLOR_INDEX.YELLOW, WD_COLOR_INDEX.YELLOW, WD_COLOR_INDEX.RED, 
            WD_COLOR_INDEX.GREEN, WD_COLOR_INDEX.BLUE, WD_COLOR_INDEX.VIOLET, 
            WD_COLOR_INDEX.DARK_RED]

# colorMap = [MSO_THEME_COLOR.ACCENT_6, MSO_THEME_COLOR.ACCENT_4, MSO_THEME_COLOR.ACCENT_5, 
#             MSO_THEME_COLOR.ACCENT_6, MSO_THEME_COLOR.ACCENT_3, MSO_THEME_COLOR.ACCENT_2, 
#             MSO_THEME_COLOR.ACCENT_1]

colorMap = [RGBColor(0x00, 0x00, 0x00), RGBColor(0xff, 0xcc, 0x99), RGBColor(0xf0, 0x00, 0xf0), 
            RGBColor(0x00, 0x33, 0xff), RGBColor(0xff, 0xa5, 0x00), RGBColor(0xff, 0x00, 0x00),
            RGBColor(0x40, 0xf0, 0x40), RGBColor(0x00, 0x00, 0x00), RGBColor(0x00, 0x00, 0x00),
            RGBColor(0x42, 0xc5, 0xf5)]

def splitTxt(arrTxt, key):
    newArrTxt = []
    for sent in arrTxt:
        if key in sent:
            arrSplit = sent.split(key)

            slen = len(arrSplit)
#             print(key, "==>split:", arrSplit)
            word = arrSplit[0]
            for idx in range(1, slen):
                we = arrSplit[idx]

                if len(word) > 0 and isEnglish(word[-1]):
                    word += key
                    word += we
                    if idx == slen - 1:
                        newArrTxt.append(word)
                    continue
                elif len(we) > 0 and isEnglish(we[0]):
                    word += key
                    word += we
                    if idx == slen - 1:
                        newArrTxt.append(word)
                    continue
                
                if idx == slen - 1:
#                     print("last:", word, key, we)
                    if (len(word) > 0 and isEnglish(word[-1])) or (len(we) > 0 and isEnglish(we[0])):
                        word += key
                        word += we
                        newArrTxt.append(word)
                        continue
                    else:
                        newArrTxt.append(word)
                        newArrTxt.append(key)
                        newArrTxt.append(we)
                        continue
                
                newArrTxt.append(word)
                newArrTxt.append(key)
                word = we

        else:
            newArrTxt.append(sent)
    return newArrTxt
    
def highlight_run(plainTxt, wordList, outputFile):
    arrPara = []
#     print(wordList)
    for paraTxt in plainTxt:
        arrTxt = [paraTxt]
#         print(arrTxt)
        for hword in wordList:
#             print("hword: ", hword)
            arrTxt = splitTxt(arrTxt, hword)
#             print(hword, arrTxt)
        arrPara.append(arrTxt)
#     print(arrPara)
    document = Document()
    
    tableItems = {}
    highlightCnt = 1
    for para in arrPara:
        docPara = document.add_paragraph("")

        for partTxt in para:              
            if partTxt in wordList:
                # get the level of the word
                level = wordList[partTxt]['level']
                lemma = wordList[partTxt]['lemma']
                if lemma not in tableItems:
                    tableItems[lemma] = level
                    
                highlightCnt += 1
#                 print(partTxt, level)
    
#                 docPara.add_run(partTxt).font.highlight_color = colorMap[level]
                run = docPara.add_run(partTxt)
                run.font.color.rgb = colorMap[level]
                if level in highlevels:
                    run.font.highlight_color = WD_COLOR_INDEX.YELLOW
            
            else:
                docPara.add_run(partTxt)

    # add table ------------------
    table = document.add_table(1, 4)

    # populate header row --------
    heading_cells = table.rows[0].cells
    heading_cells[0].text = ''
    heading_cells[1].text = 'vocab'
    heading_cells[2].text = 'level'
    heading_cells[3].text = 'Chinese'

    # sort table items
    sortedItems = sorted(tableItems.items(), key=lambda kv: kv[1], reverse=True)
#     print(sortedItems)
    
    # add a data row for each item
    # ('accord', {'idx': 1, 'level': 6})
    idx = 1
    for vocab, level in sortedItems:
#         print(vocab)
        cells = table.add_row().cells
        cells[0].text = str(idx)

        run1 = cells[1].paragraphs[0].add_run(vocab)
        run1.font.color.rgb = colorMap[level]
        if level in highlevels:
            run1.font.highlight_color = WD_COLOR_INDEX.YELLOW
            
        cells[2].paragraphs[0].add_run(str(level)).font.color.rgb = colorMap[level]
        cells[3].text = ''
    
        idx += 1
    table.style = 'Table Grid'
    
    print(highlightCnt)
    document.save(outputFile)

In [6]:
def splitWords(sDf):
    wObjs = {}
    #     index  count  level      words
    # 0   more      3      1     More;more
    for idx in range(0, len(sDf)):
        lemma, count, level, words = sDf.loc[idx]
        spWords = words.split(';')
        for w in spWords:
            if w in wObjs:
                print("ERROR!! w in wObjs:", w)
                exit
            wObjs[w] = {'lemma': lemma, 'level': level, 'count': count}
    return wObjs

In [7]:
def parseFile(readf, writeDir, sheetDf):
    sp = readf.split('/')
    if len(sp) < 2:
        print("ERROR!!", sp)
        exit
    
    # create new folder if not exist
    createDir = '{0}/{1}'.format(writeDir, readf.split('/')[-2])
    if not os.path.exists(createDir):
        os.makedirs(createDir)

    writeFile = os.path.join(createDir, readf.split('/')[-1])
#     readFile = os.path.join(root, file)
    print(readf)

    wordCnt, plainTxt, wordList = readTxt(readf)
    wordList = splitWords(sheetDf)
    highlight_run(plainTxt, wordList, writeFile)

In [8]:
# To read the localtion of all original files
oriFiles = []
def readDir(readDir):
    tmpFiles = []
    for root, dirs, files in os.walk(readDir):
        for file in sorted(files):
            # '.filename.docx': temporary file in Mac
            # '~': The file could be temporary opened.
            if '.docx' in file and file[0] != '.' and file[0] != '~':
                readFile = os.path.join(root, file)
                tmpFiles.append(readFile)
    return tmpFiles

oriFiles = readDir(from_dataPath)

In [9]:
from_xlsx = ['GSAT_multiple_20190916_v1.xlsx', 'GSAT_cloze_20190916_v1.xlsx', 'GSAT_reading_20190916_v1.xlsx',
    'Exam_multiple_20190916_v1.xlsx', 'Exam_cloze_20190916_v1.xlsx', 'Exam_discourse_20190916_v1.xlsx',
    'Exam_reading_20190916_v1.xlsx']

# from_xlsx = ['test.xlsx']
for xlsx in from_xlsx:
    fileXlsx = '{0}/{1}/{2}'.format(basePath, outputPath, xlsx)

    with pd.ExcelFile(fileXlsx) as reader:
        # read sheet by sheet
        for sheet in reader.sheet_names:
#             print(sheet)
            sheetDf = pd.read_excel(reader, sheet)
            sheetDf = sheetDf.fillna(0)
            workDf = sheetDf[sheetDf['level'].isin(levels)]
            workDf.reset_index(drop=True, inplace=True)
#             print(workDf.head())

            fileloc = [x for x in oriFiles if sheet in x]
            
            if len(fileloc) != 1:
                print("ERROR!!", fileloc)
                exit

            f = fileloc[0]
            parseFile(f, to_dataPath, workDf)

/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q16to20-M清.docx
88%  includes un-alpha characters.
55
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q21to25- M清.docx
49
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-099-Q26to30- M清.docx
2  includes un-alpha characters.
5  includes un-alpha characters.
2006  includes un-alpha characters.
90%  includes un-alpha characters.
55
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q16to20- M清.docx
42
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q21to25- M清.docx
23  includes un-alpha characters.
1999  includes un-alpha characters.
30  includes un-alpha characters.
47
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-100-Q26to30- M清.docx
46
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 1 Multiple/A-101-Q16to20- M清.docx
7  includes un-alpha 

84
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q45to48-R清.docx
1890s  includes un-alpha characters.
1920s  includes un-alpha characters.
1930s  includes un-alpha characters.
1960s  includes un-alpha characters.
1980s  includes un-alpha characters.
1990s  includes un-alpha characters.
105
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q49to52-R清.docx
1953  includes un-alpha characters.
1992  includes un-alpha characters.
28  includes un-alpha characters.
000  includes un-alpha characters.
30%  includes un-alpha characters.
US$100  includes un-alpha characters.
600  includes un-alpha characters.
98
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-099- Q53to56-R清.docx
10  includes un-alpha characters.
000  includes un-alpha characters.
18  includes un-alpha characters.
12  includes un-alpha characters.
$9  includes un-alpha characters.
10  includes un-alpha characters.
000  includes un-alpha 

99
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-107- Q53to56-R清.docx
89
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q41to44-R清.docx
79
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q45to48-R清.docx
145
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q49to52-R清.docx
88
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/學測 3 Reading/A-108- Q53to56-R清.docx
102
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-099-Q11to15- M 清.docx
20  includes un-alpha characters.
000  includes un-alpha characters.
52
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-099-Q16to20- M 清.docx
15  includes un-alpha characters.
2008  includes un-alpha characters.
54
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 1 multiple/B-100-Q11to15- M 清.docx
59
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文

96
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q40to43- R 清.docx
18  includes un-alpha characters.
24  includes un-alpha characters.
10  includes un-alpha characters.
15  includes un-alpha characters.
24  includes un-alpha characters.
2007  includes un-alpha characters.
20  includes un-alpha characters.
90
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q44to47- R 清.docx
$50  includes un-alpha characters.
500  includes un-alpha characters.
44  includes un-alpha characters.
132
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-099- Q48to51- R 清.docx
1999  includes un-alpha characters.
1996  includes un-alpha characters.
122
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-100- Q36to39- R 清.docx
US$250  includes un-alpha characters.
US$450  includes un-alpha characters.
101
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-100- Q40to43- R 清.

109
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-106- Q44to47- R 清.docx
2009  includes un-alpha characters.
1996  includes un-alpha characters.
176
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-106- Q48to51- R 清.docx
16th  includes un-alpha characters.
1740s  includes un-alpha characters.
18th  includes un-alpha characters.
19th  includes un-alpha characters.
1860s  includes un-alpha characters.
Ōyama  includes un-alpha characters.
19th  includes un-alpha characters.
169
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-107- Q36to39- R 清.docx
1785  includes un-alpha characters.
19th  includes un-alpha characters.
12  includes un-alpha characters.
1817  includes un-alpha characters.
13  includes un-alpha characters.
60  includes un-alpha characters.
114
/Volumes/backup_128G/z_repository/Yang_data/學測指考英文/data_docx/指考 4 reading/B-107- Q40to43- R 清.docx
2011  includes un-alpha characters.
224  includes u